<a href="https://colab.research.google.com/github/diaskrv/deepl/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [201]:
import pandas as pd
import numpy as np
import tensorflow as tf

from scipy.sparse import csr_matrix

from sklearn.neighbors import NearestNeighbors

In [202]:
movies_info = pd.read_csv('/content/movies.csv')
ratings_info = pd.read_csv('/content/ratings.csv')

In [203]:
movies_info.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [204]:
ratings_info.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [205]:
rating_by_movie_list = ratings_info.pivot(index = 'movieId', columns = 'userId', values= 'rating')
rating_by_movie_list.fillna(0, inplace = True)
rating_by_movie_list.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [206]:
number_of_votes_by_user = ratings_info.groupby('userId')['rating'].agg('count')
number_of_votes_of_movie = ratings_info.groupby('movieId')['rating'].agg('count')

user_mask = number_of_votes_by_user[number_of_votes_by_user > 40].index
movie_mask = number_of_votes_of_movie[number_of_votes_of_movie > 10].index

rating_by_movie_list = rating_by_movie_list.loc[movie_mask,:]
rating_by_movie_list = rating_by_movie_list.loc[:,user_mask]

rating_by_movie_list.shape

(2121, 422)

In [207]:
rating_by_movie_list = rating_by_movie_list.rename_axis(None, axis = 1).reset_index()
rating_by_movie_list.head()

,movieId,1,4,5,6,7,8,9,10,11,...,600,601,602,603,604,605,606,607,608,610
0,1,4.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,0.0,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
1,2,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
2,3,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,5,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
4,6,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0


In [208]:
csr_data = csr_matrix(rating_by_movie_list.values)

Model training

In [209]:
knn = NearestNeighbors(n_neighbors = 20)

knn.fit(csr_data)

NearestNeighbors(n_neighbors=20)

In [210]:
rec_number = 15

searched_movie = 'Ant-Man'

In [211]:
movie_search = movies_info[movies_info['title'].str.contains(searched_movie)]
movie_search

,movieId,title,genres
8689,122900,Ant-Man (2015),Action|Adventure|Sci-Fi
9713,188301,Ant-Man and the Wasp (2018),Action|Adventure|Comedy|Fantasy|Sci-Fi


In [212]:
movie_id = movie_search.iloc[0]['movieId']

movie_id = rating_by_movie_list[rating_by_movie_list['movieId'] == movie_id].index[0]
movie_id

2079

In [213]:
distances, indices = knn.kneighbors(csr_data[movie_id], n_neighbors = rec_number + 1)

print(indices)
print(distances)

[[2079 2081 2078 2082 2080 2083 2077 2085 2084 2086 2076 2087 2088 2075
  2074 2073]]
[[   0.           17.25543393   19.21587885   20.80865205   21.63908501
    23.42541355   23.59555043   24.50510151   25.46566316   27.84780063
    27.87471973   27.97766967   30.28200786 3755.04607162 3759.05194165
  4204.04222148]]


In [214]:
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()

indices_distances = list(zip(indices_list, distances_list))

print(indices_distances[:5])

[(2079, 0.0), (2081, 17.255433926737396), (2078, 19.215878850575635), (2082, 20.808652046684813), (2080, 21.639085008382402)]


In [215]:
indices_distances = sorted(indices_distances, key = lambda x: x[1])

indices_distances = indices_distances[1:]
indices_distances

[(2081, 17.255433926737396),
 (2078, 19.215878850575635),
 (2082, 20.808652046684813),
 (2080, 21.639085008382402),
 (2083, 23.425413550244958),
 (2077, 23.595550427993835),
 (2085, 24.50510150968569),
 (2084, 25.465663156493687),
 (2086, 27.847800631288642),
 (2076, 27.874719729532707),
 (2087, 27.977669667075563),
 (2088, 30.28200785945344),
 (2075, 3755.046071621492),
 (2074, 3759.0519416469892),
 (2073, 4204.042221481606)]

In [216]:
recom_list = []

for ind_dist in indices_distances:

    matrix_movie_id = rating_by_movie_list.iloc[ind_dist[0]]['movieId']
    id = movies_info[movies_info['movieId'] == matrix_movie_id].index
    title = movies_info.iloc[id]['title'].values[0]
    dist = ind_dist[1]

    recom_list.append({'Title' : title, 'Distance' : dist})

In [217]:
recommendation_table = pd.DataFrame(recom_list, index = range(1, rec_number + 1))
recommendation_table

,Title,Distance
1,Black Panther (2017),17.255434
2,Avengers: Age of Ultron (2015),19.215879
3,Avengers: Infinity War - Part I (2018),20.808652
4,Deadpool (2016),21.639085
5,Thor: Ragnarok (2017),23.425414
6,Star Wars: Episode VII - The Force Awakens (2015),23.595550
7,Captain America: Civil War (2016),24.505102
8,Guardians of the Galaxy 2 (2017),25.465663
9,Doctor Strange (2016),27.847801
10,Mad Max: Fury Road (2015),27.874720


In [218]:
genre_labels = set()
for s in movies_info['genres'].str.split('|').values:
    genre_labels = genre_labels.union(set(s))
genre_labels

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [219]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies_info['genres'])

In [220]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

array([[1.        , 0.31379419, 0.0611029 , 0.05271111],
       [0.31379419, 1.        , 0.        , 0.        ],
       [0.0611029 , 0.        , 1.        , 0.35172407],
       [0.05271111, 0.        , 0.35172407, 1.        ]])

In [221]:
titles = movies_info['title']
genres = movies_info['genres']
indices = pd.Series(movies_info.index, index=movies_info['title'])
indices2 = pd.Series(movies_info.index, index=movies_info['genres'])

In [222]:
def similar_genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=False)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]
    
    
similar_genre_recommendations('Jumanji (1995)').head(10)

3               Waiting to Exhale (1995)
4     Father of the Bride Part II (1995)
5                            Heat (1995)
6                         Sabrina (1995)
8                    Sudden Death (1995)
10        American President, The (1995)
11    Dracula: Dead and Loving It (1995)
13                          Nixon (1995)
15                         Casino (1995)
16          Sense and Sensibility (1995)
Name: title, dtype: object